In [28]:
import pandas as pd
import json
from pathlib import Path

df = pd.read_csv("../data/input/train_metadata.csv")
df.sample(5)

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,date,filename,license,rating,time,url
47656,savspa,['comyel'],"['life stage uncertain', 'male', 'song']",25.6007,-109.0504,Passerculus sandwichensis,Savannah Sparrow,Manuel Grosselet,2020-03-04,XC531828.ogg,Creative Commons Attribution-NonCommercial-Sha...,4.5,08:30,https://www.xeno-canto.org/531828
25717,greegr,[],"['call', 'flight call']",-33.5608,150.8196,Ardea alba,Great Egret,Greg McLachlan,2017-04-22,XC365950.ogg,Creative Commons Attribution-NonCommercial-Sha...,4.5,16:30,https://www.xeno-canto.org/365950
55920,wbwwre1,[],['song'],-0.7120,-77.7410,Henicorhina leucosticta,White-breasted Wood-Wren,Niels Krabbe,1993-10-11,XC242758.ogg,Creative Commons Attribution-ShareAlike 4.0,5.0,06:30:00 AM,https://www.xeno-canto.org/242758
52491,sumtan,[],"['male', 'song']",41.7425,-111.8184,Piranga rubra,Summer Tanager,Andrew Durso,2015-07-01,XC254864.ogg,Creative Commons Attribution-NonCommercial-Sha...,4.0,20:24,https://www.xeno-canto.org/254864
30488,kebtou1,[],['song'],17.2210,-89.6230,Ramphastos sulfuratus,Keel-billed Toucan,eloisa matheu,1994-03-15,XC313139.ogg,Creative Commons Attribution-NonCommercial-Sha...,4.0,07:00,https://www.xeno-canto.org/313139


In [29]:
output = Path("../data/metadata")
output.mkdir(exist_ok=True, parents=True)

In [31]:
for row in df[["primary_label", "scientific_name", "common_name"]].drop_duplicates().to_dict(orient="records"):
    p = output / row["primary_label"]
    p.mkdir(exist_ok=True)
    (p / "info.json").write_text(json.dumps(row))

In [41]:
fields = ["name", "secondary_labels", "type", "date", "latitude", "longitude", "rating", "url"]
df["name"] = df.filename.str.rstrip(".ogg")
df[df.primary_label == "acafly"][fields]

,name,secondary_labels,type,date,latitude,longitude,rating,url
0,XC109605,['amegfi'],"['begging call', 'call', 'juvenile']",2012-08-12,35.3860,-84.1250,2.5,https://www.xeno-canto.org/109605
1,XC11209,[],['call'],2000-12-26,9.1334,-79.6501,3.0,https://www.xeno-canto.org/11209
2,XC127032,[],['call'],2012-01-10,5.7813,-75.7452,3.0,https://www.xeno-canto.org/127032
3,XC129974,['whwbec1'],['call'],2009-06-19,4.6717,-75.6283,3.5,https://www.xeno-canto.org/129974
4,XC129981,['whwbec1'],['call'],2009-06-19,4.6717,-75.6283,3.5,https://www.xeno-canto.org/129981
...,...,...,...,...,...,...,...,...
127,XC580740,[],['call'],2020-08-02,40.8918,-91.0756,4.0,https://www.xeno-canto.org/580740
128,XC583856,"['reevir1', 'yetvir', 'robgro', 'amerob']","['adult', 'male', 'song']",2020-06-09,43.2167,-81.9021,4.5,https://www.xeno-canto.org/583856
129,XC594106,[],"['call', 'life stage uncertain', 'sex uncertain']",2020-10-09,31.9237,-81.0559,5.0,https://www.xeno-canto.org/594106
130,XC600277,"['whbnut', 'indbun']","['life stage uncertain', 'sex uncertain', 'song']",2020-07-24,43.0989,-71.0524,3.5,https://www.xeno-canto.org/600277


In [49]:
for label in df.primary_label.drop_duplicates():
    p = output / label
    p.mkdir(exist_ok=True)
    res = []
    for row in df[df.primary_label == label][fields].to_dict(orient="records"):
        for i in ["secondary_labels", "type"]:
            row[i] = eval(row[i])
        res.append(row)
    (p / "metadata.json").write_text(json.dumps(res, indent=2))